## Part 1: game with elves fighting goblins

In [3]:
day15input=open("input.txt")
input = day15input.readlines()

In [13]:
len(input), len(input[0].strip())

(32, 32)

In [6]:
input

['################################\n',
 '##############.#################\n',
 '##############...###############\n',
 '#..#######G.#....###############\n',
 '##.#####.#G....#.###############\n',
 '##..###.......##################\n',
 '###.####..G.G.##################\n',
 '###.##G.......##################\n',
 '###.G.....#...G######.##########\n',
 '###GGG..#......####E...#########\n',
 '###.GG.....########..#..#..#####\n',
 '#########.G...G.........#....###\n',
 '#########.....#####............#\n',
 '########...G.#######.......##..#\n',
 '########....#########...#.####.#\n',
 '######E#.G..#########.....#....#\n',
 '######....E.#########....##...##\n',
 '######......#########.....##..##\n',
 '######......#########...########\n',
 '#..###.......#######.....#######\n',
 '#..G##.G......#####..G...#######\n',
 '#.........#..E...........#######\n',
 '####.......E............########\n',
 '####.#..E.G..............#######\n',
 '###########..............#######\n',
 '###########............

In [14]:
#will read in board, positions of creatures
#need a creature class first

In [489]:
def in_bounds(pos,board):
    return(pos[0] >= 0 and pos[0] < len(board[0]) and pos[1] >= 0 and pos[1] < len(board))

def adjacents(pos,board):
    locations=[]
    #reading order: up left right down
    potentials = [(pos[0],pos[1]-1),(pos[0]-1,pos[1]),(pos[0]+1,pos[1]),(pos[0],pos[1]+1)]
    for potential in potentials:
        #check in bound
        if in_bounds(potential,board):
            locations.append((potential,board[potential[1]][potential[0]]))
        else:
            locations.append((potential,'0'))
    return(locations)

def x_redundant_find_path(pos,board,targets):
    #explore potential paths until a target found
    found = False
    paths = [[pos]]
    while not found and len(paths) > 0:
        #if len(paths) in (5,10,15,20):
              #print(paths)
        paths_next = []
        #extend possible paths then check for matches
        for path in paths:
            #print(len(path))
            latest = path[-1]
            steps = [adjacent[0] for adjacent in adjacents(latest,board) if adjacent[1] == '.' and adjacent[0] not in path]
            #print(steps, paths_next)
            for step in steps:
                #if no valid steps then this path won't appear any more
                new_path = path[:]
                new_path.append(step)
                paths_next.append(new_path)
                if step in targets:
                    found = True
        paths = paths_next[:]
    #either no paths or have a list of paths of same length with one or more matches
    if len(paths) == 0:
        #no move
        return(pos)
    else:
        found_targets = [target for target in targets if target in [path[-1] for path in paths]]
        found_targets.sort(key = lambda coord: (coord[1],coord[0]))
        chosen = found_targets[0]
        target_paths = list(filter(lambda x: x[-1] == chosen , paths))
        #print(target_paths)
        target_paths.sort(key= lambda path: (path[1][1],path[1][0]))
        return(target_paths[0][1])
    

In [490]:
def find_path(pos,board,targets):
    #explore potential paths until a target found
    #only explore each square once
    found = False
    points = [set([pos])]
    while not found and len(points[-1]) > 0:
        points_next = set([])
        #extend possible paths then check for matches
        for latest_point in points[-1]:
            if len(points) > 1:
                steps = set([adjacent[0] for adjacent in adjacents(latest_point,board) if adjacent[1] == '.' and adjacent[0] not in points[-2]])
            else:
                steps = set([adjacent[0] for adjacent in adjacents(latest_point,board) if adjacent[1] == '.'])
            points_next |= steps
        if len(points_next.intersection(targets)) > 0:
            found = True
        points.append(points_next)
    #either no paths or have a list of points of same distance with one or more matches
    if len(points[-1]) == 0:
        #no move
        return(pos)
    else:
        found_targets = list(points_next.intersection(targets))
        found_targets.sort(key = lambda coord: (coord[1],coord[0]))
        chosen = found_targets[0]
        #print(pos, 'chose target ', chosen)
        #find first steps for shortest paths to this target
        if chosen in points[1]:
            #just one step away
            return(chosen)
        chosen_prev = set(adjacent[0] for adjacent in adjacents(chosen,board)).intersection(points[-2])
        for distance in range(-3,-len(points),-1):
            prev_prev = set([])
            for prev in chosen_prev:
                prev_prev |= set([adjacent[0] for adjacent in adjacents(prev,board)])
            chosen_prev = prev_prev.intersection(points[distance])
        target_paths = list(chosen_prev)
        #print('potential steps: ', target_paths, points)
        target_paths.sort(key= lambda path: (path[1],path[0]))
        return(target_paths[0])
    

In [454]:
list(range(-3,-10,-1))

[-3, -4, -5, -6, -7, -8, -9]

In [701]:
class creature(object):
    def __init__(self,variety,x,y,attack=3):
        self.variety = variety #Elf or Goblin
        self.x = x
        self.y = y
        self.alive = True
        self.attack = attack
        self.health = 200
        
    def in_range(self,board):
        #find adjacent spaces available
        return([adjacent[0] for adjacent in adjacents((self.x,self.y),board) if adjacent[1] == '.'])
    
    def move(self,creatures,board):
        #find living creatures of opposite variety
        for adjacent in adjacents((self.x,self.y),board):
            #returns in reading order so ok to attack first
            if adjacent[1] not in ['.','#',self.variety]:
                self.attack_adjacent(board,creatures)
                return(0)
        #if not one adjacent, search for one
        targets = list(filter(lambda x: x.variety != self.variety and x.alive == True, creatures))
        #if no targets, game over
        if len(targets) == 0:
            return(1)
        #filter those with space in range
        valid_targets = set([])
        for target in targets:
            valid_targets |= set(target.in_range(board))
        #find closest in range with path to reach
        next_pos = find_path((self.x,self.y),board,valid_targets)
        #print('new_position = ', next_pos)
        #move toward closest 
        board[self.y][self.x] = '.'
        board[next_pos[1]][next_pos[0]] = self.variety
        self.x,self.y = next_pos[0], next_pos[1]
        self.attack_adjacent(board,creatures)
        return(0)
    
    def attack_adjacent(self,board,creatures):
        potentials = []
        for adjacent in adjacents((self.x,self.y),board):
            #returns in reading order so ok to attack first if tie for hitpoints
            if adjacent[1] not in ['.','#',self.variety]:
                for creature in creatures:
                    #print(creature.x,creature.y,creature.alive)
                    if adjacent[0] == (creature.x,creature.y) and creature.alive:
                        potentials.append(creature)
        if len(potentials) > 0:
            potentials.sort(key= lambda creature: (creature.health,creature.y,creature.x))
            creature = potentials[0]
            #print('attack on ', creature.x, creature.y)
            creature.health -= self.attack
            if creature.health <= 0:
                creature.alive = False
                print(creature.variety, 'dead at ', creature.x, creature.y)
                board[creature.y][creature.x] = '.'
        return(0)

    

    

In [676]:
board = []
creatures = []
for ii, line in enumerate(input):
    board.append([])
    for jj, char in enumerate(line.strip()):
        if char in ('E','G'):
            creatures.append(creature(char,jj,ii))
            #board[ii].append('.')
        board[ii].append(char) 
    

In [677]:
#keep track of how many full turns have passed
turns = 0

In [679]:
def move_all(board,creatures):
    creatures.sort(key= lambda creature: (creature.y,creature.x))
    no_targets = 0
    for creature in creatures:
        #print(creature.x,creature.y)
        if creature.alive == True:
            no_targets = max(no_targets,creature.move(creatures,board))
    return(no_targets)


In [680]:
no_targets = False
while not no_targets:
    no_targets = move_all(board,creatures)
    turns += 1
    print(turns)
    print([(creature.variety, creature.x, creature.y, creature.health) for creature in creatures])
    
    

attack on  10 16
attack on  10 15
attack on  10 23
attack on  10 23
attack on  10 22
1
[('G', 11, 3, 200), ('G', 10, 5, 200), ('G', 9, 6, 200), ('G', 11, 6, 200), ('G', 7, 7, 200), ('G', 5, 8, 200), ('G', 13, 8, 200), ('G', 3, 9, 200), ('G', 4, 9, 200), ('G', 6, 9, 200), ('E', 19, 10, 200), ('G', 4, 10, 200), ('G', 6, 10, 200), ('G', 10, 12, 200), ('G', 15, 11, 200), ('G', 10, 13, 200), ('E', 6, 16, 200), ('G', 10, 15, 197), ('E', 10, 16, 197), ('G', 3, 21, 200), ('G', 8, 20, 200), ('G', 22, 20, 200), ('E', 12, 21, 200), ('E', 10, 22, 197), ('E', 9, 23, 200), ('G', 10, 23, 194), ('E', 24, 24, 200), ('E', 22, 27, 200), ('E', 27, 27, 200), ('E', 26, 28, 200)]
attack on  10 16
attack on  10 15
attack on  10 23
attack on  10 23
attack on  10 22
2
[('G', 11, 4, 200), ('G', 10, 6, 200), ('G', 9, 7, 200), ('G', 11, 7, 200), ('G', 8, 7, 200), ('G', 6, 8, 200), ('G', 12, 8, 200), ('G', 3, 9, 200), ('G', 4, 9, 200), ('G', 7, 9, 200), ('G', 4, 10, 200), ('G', 7, 10, 200), ('E', 19, 11, 200), ('G'

attack on  10 16
attack on  10 15
attack on  10 16
attack on  11 20
attack on  10 20
attack on  23 22
attack on  10 22
attack on  10 23
attack on  23 22
attack on  23 23
attack on  10 23
attack on  10 22
attack on  23 22
21
[('G', 11, 4, 200), ('G', 10, 6, 200), ('G', 7, 7, 200), ('G', 3, 8, 200), ('G', 5, 9, 200), ('G', 10, 9, 200), ('G', 12, 9, 200), ('G', 4, 10, 200), ('G', 7, 10, 200), ('G', 9, 11, 200), ('G', 17, 11, 143), ('E', 18, 11, 146), ('G', 10, 12, 200), ('G', 8, 14, 146), ('E', 8, 15, 104), ('G', 9, 15, 200), ('G', 10, 15, 137), ('E', 10, 16, 107), ('G', 11, 16, 200), ('G', 10, 20, 143), ('E', 11, 20, 146), ('E', 15, 21, 200), ('E', 23, 21, 200), ('G', 9, 22, 200), ('E', 10, 22, 95), ('E', 22, 22, 200), ('G', 23, 22, 77), ('E', 9, 23, 200), ('G', 10, 23, 74), ('E', 23, 23, 146)]
attack on  18 11
attack on  17 11
attack on  8 15
attack on  8 14
attack on  8 15
attack on  10 16
attack on  10 15
attack on  10 16
attack on  11 20
attack on  10 20
attack on  23 22
attack on  1

attack on  8 15
attack on  8 14
attack on  8 15
attack on  10 16
attack on  10 15
attack on  10 16
attack on  11 20
attack on  10 20
attack on  10 22
attack on  9 22
attack on  9 22
37
[('G', 11, 4, 200), ('G', 10, 6, 200), ('G', 7, 7, 200), ('G', 3, 8, 200), ('G', 5, 9, 200), ('G', 10, 9, 200), ('G', 12, 9, 200), ('G', 4, 10, 200), ('G', 7, 10, 200), ('G', 9, 11, 200), ('G', 17, 11, 95), ('E', 18, 11, 98), ('G', 10, 12, 200), ('G', 8, 14, 98), ('E', 8, 15, 8), ('G', 9, 15, 200), ('G', 10, 15, 89), ('E', 10, 16, 11), ('G', 11, 16, 200), ('E', 11, 19, 200), ('G', 10, 20, 95), ('E', 11, 20, 98), ('E', 19, 21, 200), ('G', 9, 22, 164), ('E', 10, 22, 17), ('E', 18, 22, 200), ('G', 23, 22, -1), ('E', 9, 23, 200), ('G', 10, 23, -1), ('E', 18, 23, 122)]
attack on  18 11
attack on  17 11
attack on  8 15
attack on  8 14
attack on  8 15
attack on  10 16
attack on  10 15
attack on  10 16
attack on  10 20
attack on  11 20
attack on  10 20
attack on  10 22
attack on  9 22
attack on  9 22
38
[('G', 1

attack on  9 23
attack on  9 22
attack on  9 23
attack on  8 23
51
[('G', 10, 5, 200), ('G', 7, 7, 200), ('G', 5, 8, 200), ('G', 5, 9, 200), ('G', 9, 9, 200), ('G', 10, 10, 200), ('G', 17, 11, 53), ('E', 18, 11, 56), ('G', 9, 13, 200), ('G', 11, 13, 200), ('E', 8, 15, -1), ('G', 10, 15, 200), ('G', 7, 16, 200), ('E', 10, 16, -1), ('G', 8, 17, 200), ('G', 10, 18, 200), ('G', 9, 19, 200), ('E', 10, 19, 137), ('G', 11, 19, 68), ('E', 12, 19, 200), ('G', 10, 20, 11), ('E', 11, 20, 32), ('G', 9, 22, 98), ('E', 10, 22, -1), ('E', 10, 22, 200), ('G', 23, 22, -1), ('G', 8, 23, 83), ('E', 9, 23, 164), ('G', 10, 23, -1), ('E', 11, 23, 122)]
attack on  18 11
attack on  17 11
attack on  10 19
attack on  10 19
attack on  10 20
attack on  11 20
attack on  11 19
attack on  11 20
attack on  10 20
attack on  9 23
attack on  9 22
attack on  9 23
attack on  8 23
52
[('G', 10, 5, 200), ('G', 7, 7, 200), ('G', 5, 8, 200), ('G', 5, 9, 200), ('G', 9, 9, 200), ('G', 10, 10, 200), ('G', 17, 11, 50), ('E', 18, 

attack on  9 22
attack on  9 23
attack on  8 23
67
[('G', 7, 7, 200), ('G', 11, 7, 200), ('G', 7, 10, 200), ('G', 9, 10, 200), ('G', 17, 11, 5), ('E', 18, 11, 8), ('G', 9, 13, 200), ('G', 10, 14, 200), ('E', 8, 15, -1), ('G', 9, 16, 200), ('G', 10, 16, 200), ('E', 10, 16, -1), ('G', 11, 17, 185), ('G', 7, 18, 200), ('G', 10, 18, 200), ('E', 11, 18, 188), ('G', 9, 19, 200), ('E', 10, 19, 8), ('G', 11, 19, -1), ('G', 10, 20, 161), ('G', 10, 20, -1), ('E', 11, 20, -1), ('E', 11, 20, 122), ('G', 9, 22, 50), ('E', 10, 22, -1), ('E', 10, 22, 200), ('G', 23, 22, -1), ('G', 8, 23, 35), ('E', 9, 23, 68), ('G', 10, 23, -1)]
attack on  18 11
attack on  17 11
attack on  11 18
attack on  10 19
attack on  11 17
attack on  10 19
attack on  10 20
attack on  10 19
E dead at  10 19
attack on  10 20
attack on  9 23
attack on  9 22
attack on  9 23
attack on  8 23
68
[('G', 7, 7, 200), ('G', 11, 7, 200), ('G', 7, 10, 200), ('G', 9, 10, 200), ('G', 17, 11, 2), ('E', 18, 11, 5), ('G', 9, 13, 200), ('G', 10, 

attack on  11 18
attack on  9 22
attack on  9 22
attack on  9 21
91
[('G', 9, 11, 200), ('E', 14, 11, -1), ('G', 17, 11, -1), ('G', 10, 13, 200), ('G', 9, 13, 200), ('E', 8, 15, -1), ('G', 8, 16, 200), ('G', 9, 17, 200), ('E', 10, 16, -1), ('G', 11, 17, 113), ('G', 10, 18, 200), ('E', 11, 18, 41), ('G', 7, 20, 200), ('E', 10, 19, -1), ('G', 11, 19, 200), ('G', 11, 19, -1), ('G', 20, 19, 200), ('G', 10, 20, -1), ('G', 11, 21, 95), ('E', 11, 20, -1), ('E', 11, 20, -1), ('G', 8, 22, 200), ('G', 9, 21, 185), ('G', 9, 22, -1), ('E', 9, 22, 161), ('E', 10, 22, -1), ('G', 23, 22, -1), ('E', 9, 23, -1), ('G', 10, 23, -1), ('G', 10, 23, -1)]
attack on  11 18
attack on  11 18
attack on  11 17
attack on  11 18
attack on  9 22
attack on  9 22
attack on  9 21
92
[('G', 10, 11, 200), ('E', 14, 11, -1), ('G', 17, 11, -1), ('G', 9, 12, 200), ('G', 10, 12, 200), ('E', 8, 15, -1), ('G', 8, 17, 200), ('E', 10, 16, -1), ('G', 9, 18, 200), ('G', 11, 17, 110), ('G', 10, 18, 200), ('E', 11, 18, 32), ('E', 10

attack on  9 22
attack on  9 22
attack on  10 22
attack on  9 22
attack on  9 22
104
[('G', 10, 11, 200), ('E', 14, 11, -1), ('G', 17, 11, -1), ('G', 8, 13, 200), ('G', 9, 13, 200), ('E', 8, 15, -1), ('G', 7, 16, 200), ('E', 10, 16, -1), ('G', 11, 17, 101), ('G', 10, 18, 200), ('E', 11, 18, -1), ('E', 10, 19, -1), ('G', 10, 19, 200), ('G', 11, 19, -1), ('G', 10, 20, -1), ('G', 11, 20, 200), ('E', 11, 20, -1), ('E', 11, 20, -1), ('G', 9, 21, 179), ('G', 17, 21, 200), ('G', 8, 22, 200), ('G', 9, 22, -1), ('E', 9, 22, 20), ('E', 10, 22, -1), ('G', 10, 22, 62), ('G', 23, 22, -1), ('G', 9, 23, 200), ('E', 9, 23, -1), ('G', 10, 23, -1), ('G', 10, 23, -1)]
attack on  9 22
attack on  9 22
attack on  10 22
attack on  9 22
attack on  9 22
105
[('G', 10, 11, 200), ('E', 14, 11, -1), ('G', 17, 11, -1), ('G', 8, 13, 200), ('G', 9, 13, 200), ('E', 8, 15, -1), ('G', 7, 16, 200), ('E', 10, 16, -1), ('G', 11, 17, 101), ('G', 10, 18, 200), ('E', 11, 18, -1), ('E', 10, 19, -1), ('G', 10, 19, 200), ('G', 

In [681]:
#find final score
(turns-1)*sum(creature.health for creature in creatures if creature.health > 0)

245280

##Part 2

In [684]:
#find outcome of battle where elves have attack increased to just enough so they all survive

In [702]:
def battle_result(modifier):
    
    turns = 0
    
    board = []
    creatures = []
    for ii, line in enumerate(input):
        board.append([])
        for jj, char in enumerate(line.strip()):
            if char in ('E','G'):
                if char == 'E':
                    creatures.append(creature(char,jj,ii,modifier+3))
                else:
                    creatures.append(creature(char,jj,ii))
                #board[ii].append('.')
            board[ii].append(char) 
    elves = len([creature for creature in creatures if creature.variety=='E'])
    no_targets = False
    while not no_targets:
        no_targets = move_all(board,creatures)
        turns += 1
        print(modifier, turns)
        print([(creature.variety, creature.x, creature.y, creature.health) for creature in creatures])
    print('Elves remaining: ', elves - len([creature for creature in creatures if creature.variety=='E' and creature.alive]))
    return(elves - len([creature for creature in creatures if creature.variety=='E' and creature.alive]),turns,creatures)    

In [703]:
for modifier in range(200):
    elves_dead = battle_result(modifier)
    if elves_dead[0] == 0:
        print('win at modifier ', modifier)
        break


0 1
[('G', 11, 3, 200), ('G', 10, 5, 200), ('G', 9, 6, 200), ('G', 11, 6, 200), ('G', 7, 7, 200), ('G', 5, 8, 200), ('G', 13, 8, 200), ('G', 3, 9, 200), ('G', 4, 9, 200), ('G', 6, 9, 200), ('E', 19, 10, 200), ('G', 4, 10, 200), ('G', 6, 10, 200), ('G', 10, 12, 200), ('G', 15, 11, 200), ('G', 10, 13, 200), ('E', 6, 16, 200), ('G', 10, 15, 197), ('E', 10, 16, 197), ('G', 3, 21, 200), ('G', 8, 20, 200), ('G', 22, 20, 200), ('E', 12, 21, 200), ('E', 10, 22, 197), ('E', 9, 23, 200), ('G', 10, 23, 194), ('E', 24, 24, 200), ('E', 22, 27, 200), ('E', 27, 27, 200), ('E', 26, 28, 200)]
0 2
[('G', 11, 4, 200), ('G', 10, 6, 200), ('G', 9, 7, 200), ('G', 11, 7, 200), ('G', 8, 7, 200), ('G', 6, 8, 200), ('G', 12, 8, 200), ('G', 3, 9, 200), ('G', 4, 9, 200), ('G', 7, 9, 200), ('G', 4, 10, 200), ('G', 7, 10, 200), ('E', 19, 11, 200), ('G', 16, 11, 200), ('G', 9, 12, 200), ('G', 9, 13, 200), ('G', 10, 15, 194), ('E', 7, 16, 200), ('E', 10, 16, 194), ('G', 9, 20, 200), ('G', 23, 20, 200), ('G', 4, 21, 2

G dead at  10 23
0 31
[('G', 11, 4, 200), ('G', 10, 6, 200), ('G', 7, 7, 200), ('G', 3, 8, 200), ('G', 5, 9, 200), ('G', 10, 9, 200), ('G', 12, 9, 200), ('G', 4, 10, 200), ('G', 7, 10, 200), ('G', 9, 11, 200), ('G', 17, 11, 113), ('E', 18, 11, 116), ('G', 10, 12, 200), ('G', 8, 14, 116), ('E', 8, 15, 44), ('G', 9, 15, 200), ('G', 10, 15, 107), ('E', 10, 16, 47), ('G', 11, 16, 200), ('G', 10, 20, 113), ('E', 11, 20, 116), ('E', 22, 21, 200), ('G', 9, 22, 200), ('E', 10, 22, 35), ('E', 21, 22, 200), ('G', 23, 22, -1), ('E', 9, 23, 200), ('G', 10, 23, -1), ('E', 11, 23, 200), ('E', 21, 23, 122)]
0 32
[('G', 11, 4, 200), ('G', 10, 6, 200), ('G', 7, 7, 200), ('G', 3, 8, 200), ('G', 5, 9, 200), ('G', 10, 9, 200), ('G', 12, 9, 200), ('G', 4, 10, 200), ('G', 7, 10, 200), ('G', 9, 11, 200), ('G', 17, 11, 110), ('E', 18, 11, 113), ('G', 10, 12, 200), ('G', 8, 14, 113), ('E', 8, 15, 38), ('G', 9, 15, 200), ('G', 10, 15, 104), ('E', 10, 16, 41), ('G', 11, 16, 200), ('G', 10, 20, 110), ('E', 11, 20

G dead at  10 20
0 53
[('G', 10, 5, 200), ('G', 7, 7, 200), ('G', 5, 8, 200), ('G', 5, 9, 200), ('G', 9, 9, 200), ('G', 10, 10, 200), ('G', 17, 11, 47), ('E', 18, 11, 50), ('G', 9, 13, 200), ('G', 11, 13, 200), ('E', 8, 15, -1), ('G', 10, 15, 200), ('G', 7, 16, 200), ('E', 10, 16, -1), ('G', 8, 17, 200), ('G', 10, 18, 200), ('G', 9, 19, 200), ('E', 10, 19, 125), ('G', 11, 19, 62), ('E', 12, 19, 200), ('G', 10, 20, -1), ('E', 11, 20, 20), ('G', 9, 22, 92), ('E', 10, 22, -1), ('E', 10, 22, 200), ('G', 23, 22, -1), ('G', 8, 23, 77), ('E', 9, 23, 152), ('G', 10, 23, -1), ('E', 11, 23, 122)]
0 54
[('G', 9, 5, 200), ('G', 7, 8, 200), ('G', 6, 8, 200), ('G', 6, 9, 200), ('G', 9, 10, 200), ('G', 10, 11, 200), ('G', 17, 11, 44), ('E', 18, 11, 47), ('G', 9, 14, 200), ('G', 11, 14, 200), ('E', 8, 15, -1), ('G', 9, 15, 200), ('G', 7, 17, 200), ('E', 10, 16, -1), ('G', 8, 18, 200), ('G', 10, 18, 200), ('G', 9, 19, 200), ('E', 10, 19, 119), ('G', 11, 19, 53), ('E', 12, 19, 200), ('G', 10, 20, -1), (

G dead at  10 23
0 81
[('G', 8, 7, 200), ('G', 11, 8, 200), ('G', 7, 9, 200), ('E', 14, 11, -1), ('G', 17, 11, -1), ('G', 20, 11, 200), ('G', 9, 11, 200), ('G', 13, 11, 200), ('G', 9, 13, 200), ('G', 10, 13, 200), ('E', 8, 15, -1), ('G', 8, 16, 200), ('E', 10, 16, -1), ('G', 11, 17, 143), ('G', 10, 18, 200), ('E', 11, 18, 107), ('E', 10, 19, -1), ('G', 11, 19, 200), ('G', 11, 19, -1), ('G', 10, 20, 116), ('G', 10, 20, -1), ('E', 11, 20, -1), ('E', 11, 20, 47), ('G', 9, 22, 11), ('E', 10, 22, -1), ('E', 10, 22, 191), ('G', 23, 22, -1), ('E', 9, 23, -1), ('G', 10, 23, -1), ('G', 10, 23, -1)]
0 82
[('G', 9, 7, 200), ('G', 11, 9, 200), ('G', 7, 10, 200), ('G', 10, 11, 200), ('G', 12, 11, 200), ('E', 14, 11, -1), ('G', 17, 11, -1), ('G', 21, 11, 200), ('G', 9, 14, 200), ('G', 10, 14, 200), ('E', 8, 15, -1), ('G', 8, 17, 200), ('E', 10, 16, -1), ('G', 11, 17, 140), ('G', 10, 18, 200), ('E', 11, 18, 101), ('E', 10, 19, -1), ('G', 11, 19, 200), ('G', 11, 19, -1), ('G', 10, 20, 113), ('G', 10, 

0 103
[('G', 10, 11, 200), ('E', 14, 11, -1), ('G', 17, 11, -1), ('G', 8, 13, 200), ('G', 9, 13, 200), ('E', 8, 15, -1), ('G', 7, 16, 200), ('E', 10, 16, -1), ('G', 11, 17, 101), ('G', 10, 18, 200), ('E', 11, 18, -1), ('E', 10, 19, -1), ('G', 10, 19, 200), ('G', 11, 19, -1), ('G', 10, 20, -1), ('G', 11, 20, 200), ('E', 11, 20, -1), ('E', 11, 20, -1), ('G', 9, 21, 179), ('G', 17, 21, 200), ('G', 8, 22, 200), ('G', 9, 22, -1), ('E', 9, 22, 32), ('E', 10, 22, -1), ('G', 10, 22, 65), ('G', 23, 22, -1), ('G', 9, 23, 200), ('E', 9, 23, -1), ('G', 10, 23, -1), ('G', 10, 23, -1)]
0 104
[('G', 10, 11, 200), ('E', 14, 11, -1), ('G', 17, 11, -1), ('G', 8, 13, 200), ('G', 9, 13, 200), ('E', 8, 15, -1), ('G', 7, 16, 200), ('E', 10, 16, -1), ('G', 11, 17, 101), ('G', 10, 18, 200), ('E', 11, 18, -1), ('E', 10, 19, -1), ('G', 10, 19, 200), ('G', 11, 19, -1), ('G', 10, 20, -1), ('G', 11, 20, 200), ('E', 11, 20, -1), ('E', 11, 20, -1), ('G', 9, 21, 179), ('G', 17, 21, 200), ('G', 8, 22, 200), ('G', 9, 2

...

14 51
[('E', 9, 11, 173), ('G', 10, 11, 64), ('G', 11, 11, -4), ('E', 11, 11, 83), ('G', 14, 11, -4), ('G', 17, 11, -4), ('E', 18, 11, 179), ('G', 9, 12, -4), ('E', 10, 12, -1), ('E', 10, 12, 137), ('G', 11, 12, -4), ('E', 21, 12, 200), ('G', 9, 13, -4), ('G', 10, 13, -4), ('E', 11, 13, 59), ('G', 8, 14, -4), ('G', 8, 14, -4), ('G', 9, 14, -4), ('G', 11, 14, -4), ('E', 22, 14, 200), ('G', 9, 15, -4), ('G', 10, 15, -4), ('G', 10, 15, -4), ('G', 11, 16, -4), ('E', 21, 17, 200), ('G', 10, 20, -4), ('G', 9, 21, -4), ('E', 17, 21, 167), ('G', 23, 22, -4), ('G', 10, 23, -4)]
14 52
[('E', 9, 11, 173), ('G', 10, 11, 13), ('G', 11, 11, -4), ('E', 11, 11, 80), ('G', 14, 11, -4), ('G', 17, 11, -4), ('E', 18, 11, 179), ('G', 9, 12, -4), ('E', 10, 12, -1), ('E', 10, 12, 137), ('G', 11, 12, -4), ('E', 21, 12, 200), ('G', 9, 13, -4), ('G', 10, 13, -4), ('E', 11, 13, 59), ('G', 8, 14, -4), ('G', 8, 14, -4), ('G', 9, 14, -4), ('G', 11, 14, -4), ('E', 22, 14, 200), ('G', 9, 15, -4), ('G', 10, 15, -4), (

15 27
[('G', 9, 8, 200), ('G', 7, 8, 200), ('G', 11, 9, 200), ('G', 5, 10, 200), ('G', 7, 9, 200), ('G', 10, 12, 200), ('G', 14, 11, 20), ('E', 15, 11, 137), ('G', 17, 11, -16), ('G', 8, 14, 128), ('G', 8, 14, -16), ('G', 9, 14, 146), ('G', 11, 14, 164), ('E', 8, 15, 89), ('G', 9, 15, -16), ('E', 9, 15, 188), ('G', 10, 15, -16), ('G', 10, 15, -16), ('E', 10, 14, 104), ('E', 11, 15, 152), ('G', 11, 16, -16), ('E', 9, 17, 167), ('G', 10, 20, -16), ('G', 9, 21, -16), ('E', 16, 21, 200), ('E', 18, 21, 179), ('E', 16, 22, 200), ('G', 23, 22, -16), ('G', 10, 23, -16), ('E', 15, 23, 200)]
15 28
[('G', 6, 8, 200), ('G', 9, 9, 200), ('G', 7, 10, 200), ('G', 10, 9, 200), ('G', 5, 10, 200), ('G', 14, 11, 2), ('E', 15, 11, 134), ('G', 17, 11, -16), ('G', 10, 13, 200), ('G', 8, 14, 110), ('G', 8, 14, -16), ('G', 9, 14, 110), ('E', 10, 14, 95), ('G', 11, 14, 146), ('E', 8, 15, 86), ('G', 9, 15, -16), ('E', 9, 15, 188), ('G', 10, 15, -16), ('G', 10, 15, -16), ('E', 11, 15, 152), ('G', 11, 16, -16), (

[('E', 9, 11, 173), ('G', 10, 11, -16), ('G', 11, 11, -16), ('E', 11, 11, 80), ('G', 14, 11, -16), ('G', 17, 11, -16), ('E', 18, 11, 179), ('G', 9, 12, -16), ('E', 10, 12, -1), ('E', 10, 12, 137), ('G', 11, 12, -16), ('E', 21, 12, 200), ('G', 9, 13, -16), ('G', 10, 13, -16), ('E', 11, 13, 59), ('G', 8, 14, -16), ('G', 8, 14, -16), ('G', 9, 14, -16), ('G', 11, 14, -16), ('E', 22, 14, 200), ('G', 9, 15, -16), ('G', 10, 15, -16), ('G', 10, 15, -16), ('G', 11, 16, -16), ('E', 21, 17, 200), ('G', 10, 20, -16), ('G', 9, 21, -16), ('E', 17, 21, 167), ('G', 23, 22, -16), ('G', 10, 23, -16)]
Elves remaining:  1
16 1
[('G', 11, 3, 200), ('G', 10, 5, 200), ('G', 9, 6, 200), ('G', 11, 6, 200), ('G', 7, 7, 200), ('G', 5, 8, 200), ('G', 13, 8, 200), ('G', 3, 9, 200), ('G', 4, 9, 200), ('G', 6, 9, 200), ('E', 19, 10, 200), ('G', 4, 10, 200), ('G', 6, 10, 200), ('G', 10, 12, 200), ('G', 15, 11, 200), ('G', 10, 13, 200), ('E', 6, 16, 200), ('G', 10, 15, 181), ('E', 10, 16, 197), ('G', 3, 21, 200), ('G'

16 25
[('G', 10, 6, 200), ('G', 9, 8, 200), ('G', 6, 9, 200), ('G', 7, 10, 200), ('G', 10, 10, 200), ('G', 5, 10, 200), ('G', 14, 11, 29), ('E', 15, 11, 143), ('G', 17, 11, -9), ('G', 8, 14, 143), ('G', 8, 14, -9), ('G', 9, 14, 143), ('G', 10, 15, 200), ('E', 8, 15, 98), ('G', 9, 15, -9), ('E', 9, 15, 176), ('G', 10, 15, -9), ('G', 10, 15, -9), ('E', 8, 16, 179), ('E', 10, 16, 104), ('G', 11, 16, -9), ('E', 11, 16, 170), ('G', 10, 20, -9), ('G', 9, 21, -9), ('E', 15, 21, 200), ('E', 15, 22, 182), ('G', 23, 22, -9), ('G', 10, 23, -9), ('E', 14, 23, 200), ('E', 13, 24, 200)]
16 26
[('G', 11, 6, 200), ('G', 9, 9, 200), ('G', 6, 9, 200), ('G', 5, 10, 200), ('G', 8, 10, 200), ('G', 10, 11, 200), ('G', 14, 11, 10), ('E', 15, 11, 140), ('G', 17, 11, -9), ('G', 8, 14, 124), ('G', 8, 14, -9), ('G', 9, 14, 124), ('E', 8, 15, 95), ('G', 9, 15, -9), ('E', 9, 15, 173), ('G', 10, 15, 162), ('G', 10, 15, -9), ('G', 10, 15, -9), ('E', 8, 16, 179), ('E', 10, 16, 101), ('G', 11, 16, -9), ('E', 11, 15, 1

16 49
[('G', 10, 10, 143), ('G', 9, 11, -9), ('E', 9, 10, 146), ('E', 10, 11, 83), ('G', 13, 11, -9), ('G', 14, 11, -9), ('G', 17, 11, -9), ('E', 8, 13, 65), ('G', 9, 13, -9), ('G', 10, 13, -9), ('E', 10, 13, 62), ('G', 11, 13, -9), ('E', 11, 13, 137), ('G', 8, 14, -9), ('G', 8, 14, -9), ('G', 9, 14, -9), ('G', 9, 15, -9), ('G', 10, 15, -9), ('G', 10, 15, -9), ('G', 10, 15, -9), ('E', 11, 16, 179), ('G', 11, 16, -9), ('G', 10, 20, -9), ('G', 9, 21, -9), ('E', 17, 21, 200), ('E', 17, 22, 182), ('G', 23, 22, -9), ('G', 10, 23, -9), ('E', 16, 23, 200), ('E', 15, 24, 200)]
16 50
[('E', 9, 10, 146), ('G', 10, 10, 105), ('G', 9, 11, -9), ('E', 10, 11, 80), ('G', 13, 11, -9), ('G', 14, 11, -9), ('G', 17, 11, -9), ('E', 8, 13, 65), ('G', 9, 13, -9), ('G', 10, 13, -9), ('E', 10, 13, 62), ('G', 11, 13, -9), ('E', 11, 13, 137), ('G', 8, 14, -9), ('G', 8, 14, -9), ('G', 9, 14, -9), ('G', 9, 15, -9), ('G', 10, 15, -9), ('G', 10, 15, -9), ('G', 10, 15, -9), ('E', 11, 16, 179), ('G', 11, 16, -9), ('G

In [710]:
#find final score
(elves_dead[1]-1)*sum([creature.health for creature in elves_dead[2] if creature.health>0])

74984

In [709]:
turns

106

Testing follows

In [663]:
#######   
#G....#   G(200)
#.G...#   G(131)
#.#.#G#   G(59)
#...#.#   
#....G#   G(200)
#######  
test_input=['#######', '#.G...#', '#...EG#',
 '#.#.#G#',
 '#..G#E#',
 '#.....#',
 '#######']

In [670]:
#######       #######
#G..#E#       #...#E#   E(200)
#E#E.E#       #E#...#   E(197)
#G.##.#  -->  #.E##.#   E(185)
#...#E#       #E..#E#   E(200), E(200)
#...E.#       #.....#
#######
test_input=['#######', '#G..#E#', '#E#E.E#',
 '#G.##.#',
 '#...#E#',
 '#...E.#',
 '#######']

In [671]:
testboard = []
testcreatures = []
for ii, line in enumerate(test_input):
    testboard.append([])
    for jj, char in enumerate(line.strip()):
        if char in ('E','G'):
            testcreatures.append(creature(char,jj,ii))
            #board[ii].append('.')
        testboard[ii].append(char) 
    

In [672]:
testboard

[['#', '#', '#', '#', '#', '#', '#'],
 ['#', 'G', '.', '.', '#', 'E', '#'],
 ['#', 'E', '#', 'E', '.', 'E', '#'],
 ['#', 'G', '.', '#', '#', '.', '#'],
 ['#', '.', '.', '.', '#', 'E', '#'],
 ['#', '.', '.', '.', 'E', '.', '#'],
 ['#', '#', '#', '#', '#', '#', '#']]

In [673]:
print([(creature.variety, creature.x, creature.y, creature.health) for creature in testcreatures])

[('G', 1, 1, 200), ('E', 5, 1, 200), ('E', 1, 2, 200), ('E', 3, 2, 200), ('E', 5, 2, 200), ('G', 1, 3, 200), ('E', 5, 4, 200), ('E', 4, 5, 200)]


In [674]:
turns=0
no_targets = False
while not no_targets:
    no_targets = move_all(testboard,testcreatures)
    turns += 1
    print(turns)
    print([(creature.variety, creature.x, creature.y, creature.health) for creature in testcreatures])
    
    

attack on  1 2
attack on  1 1
attack on  1 2
1
[('G', 1, 1, 197), ('E', 5, 1, 200), ('E', 1, 2, 194), ('E', 3, 1, 200), ('E', 5, 2, 200), ('G', 1, 3, 200), ('E', 5, 4, 200), ('E', 3, 5, 200)]
attack on  1 2
attack on  1 1
attack on  1 1
attack on  1 2
2
[('G', 1, 1, 191), ('E', 2, 1, 200), ('E', 5, 1, 200), ('E', 1, 2, 188), ('E', 5, 2, 200), ('G', 1, 3, 200), ('E', 5, 4, 200), ('E', 3, 4, 200)]
attack on  1 2
attack on  1 1
attack on  1 1
attack on  1 2
3
[('G', 1, 1, 185), ('E', 2, 1, 200), ('E', 5, 1, 200), ('E', 1, 2, 182), ('E', 5, 2, 200), ('G', 1, 3, 200), ('E', 2, 4, 200), ('E', 5, 5, 200)]
attack on  1 2
attack on  1 1
attack on  1 1
attack on  1 2
attack on  1 3
4
[('G', 1, 1, 179), ('E', 2, 1, 200), ('E', 5, 1, 200), ('E', 1, 2, 176), ('E', 5, 2, 200), ('G', 1, 3, 197), ('E', 2, 3, 200), ('E', 4, 5, 200)]
attack on  1 2
attack on  1 1
attack on  1 1
attack on  1 2
attack on  1 3
5
[('G', 1, 1, 173), ('E', 2, 1, 200), ('E', 5, 1, 200), ('E', 1, 2, 170), ('E', 5, 2, 200), ('G'

In [675]:
turns, (turns-1)*sum(creature.health for creature in testcreatures if creature.health > 0)

(38, 36334)

In [641]:
testboard

[['#', '#', '#', '#', '#', '#', '#'],
 ['#', 'G', 'E', '.', '#', '.', '#'],
 ['#', 'E', '.', '.', '.', '.', '#'],
 ['#', '.', 'E', '#', '#', '.', '#'],
 ['#', 'E', '.', '.', '#', 'E', '#'],
 ['#', '.', '.', '.', '.', '.', '#'],
 ['#', '#', '#', '#', '#', '#', '#']]

In [564]:
#testing
move_all(testboard,testcreatures)

1

In [486]:
[(creature.x, creature.y) for creature in creatures]


[(11, 3),
 (10, 5),
 (9, 6),
 (11, 6),
 (7, 7),
 (5, 8),
 (13, 8),
 (3, 9),
 (4, 9),
 (6, 9),
 (19, 10),
 (4, 10),
 (6, 10),
 (10, 12),
 (15, 11),
 (10, 13),
 (6, 16),
 (10, 15),
 (10, 16),
 (3, 21),
 (8, 20),
 (22, 20),
 (12, 21),
 (10, 22),
 (9, 23),
 (10, 23),
 (24, 24),
 (22, 27),
 (27, 27),
 (26, 28)]

In [313]:
adjacents((10,16),board)

[((10, 15), 'G'), ((9, 16), '.'), ((11, 16), '.'), ((10, 17), '.')]

In [246]:
board[4][10]

'.'

In [247]:
input[4][10]

'G'

In [499]:
print([(creature.variety, creature.x, creature.y, creature.health) for creature in creatures])

[('G', 10, 10, 200), ('G', 11, 11, 200), ('E', 14, 11, -1), ('G', 17, 11, -1), ('G', 9, 13, 200), ('G', 8, 14, 200), ('G', 9, 14, 200), ('E', 8, 15, -1), ('E', 10, 16, -1), ('G', 9, 19, 200), ('G', 10, 19, 200), ('G', 12, 19, 170), ('E', 12, 19, -1), ('G', 11, 20, 41), ('E', 11, 20, -1), ('G', 12, 20, -1), ('E', 12, 20, -1), ('G', 13, 20, 197), ('E', 11, 21, -1), ('G', 12, 21, 170), ('G', 9, 22, -1), ('G', 9, 22, -1), ('G', 9, 22, -1), ('G', 9, 22, -1), ('E', 10, 22, -1), ('E', 10, 22, -1), ('G', 11, 22, 194), ('G', 23, 22, -1), ('E', 9, 23, -1), ('G', 10, 23, -1)]


In [234]:
testpaths = [[(10, 3), (11, 3), (11, 4), (11, 5), (10, 5), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (9, 12), (10, 12), (10, 13), (10, 14), (10, 15)], [(10, 3), (11, 3), (11, 4), (11, 5), (10, 5), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (9, 12), (9, 13), (10, 13), (10, 14), (10, 15)], [(10, 3), (11, 3), (11, 4), (11, 5), (10, 5), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (10, 14), (10, 15)], [(10, 3), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (10, 7), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (9, 12), (10, 12), (10, 13), (10, 14), (10, 15)], [(10, 3), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (10, 7), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (9, 12), (9, 13), (10, 13), (10, 14), (10, 15)], [(10, 3), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (10, 7), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (10, 14), (10, 15)], [(10, 3), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (10, 9), (9, 9), (9, 10), (9, 11), (9, 12), (10, 12), (10, 13), (10, 14), (10, 15)], [(10, 3), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (10, 9), (9, 9), (9, 10), (9, 11), (9, 12), (9, 13), (10, 13), (10, 14), (10, 15)], [(10, 3), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (10, 9), (9, 9), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (10, 14), (10, 15)], [(10, 3), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (10, 9), (10, 10), (9, 10), (9, 11), (9, 12), (10, 12), (10, 13), (10, 14), (10, 15)], [(10, 3), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (10, 9), (10, 10), (9, 10), (9, 11), (9, 12), (9, 13), (10, 13), (10, 14), (10, 15)], [(10, 3), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (10, 9), (10, 10), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (10, 14), (10, 15)]]

In [235]:
testpaths

[[(10, 3),
  (11, 3),
  (11, 4),
  (11, 5),
  (10, 5),
  (9, 5),
  (9, 6),
  (9, 7),
  (9, 8),
  (9, 9),
  (9, 10),
  (9, 11),
  (9, 12),
  (10, 12),
  (10, 13),
  (10, 14),
  (10, 15)],
 [(10, 3),
  (11, 3),
  (11, 4),
  (11, 5),
  (10, 5),
  (9, 5),
  (9, 6),
  (9, 7),
  (9, 8),
  (9, 9),
  (9, 10),
  (9, 11),
  (9, 12),
  (9, 13),
  (10, 13),
  (10, 14),
  (10, 15)],
 [(10, 3),
  (11, 3),
  (11, 4),
  (11, 5),
  (10, 5),
  (9, 5),
  (9, 6),
  (9, 7),
  (9, 8),
  (9, 9),
  (9, 10),
  (9, 11),
  (9, 12),
  (9, 13),
  (9, 14),
  (10, 14),
  (10, 15)],
 [(10, 3),
  (11, 3),
  (11, 4),
  (11, 5),
  (11, 6),
  (11, 7),
  (10, 7),
  (9, 7),
  (9, 8),
  (9, 9),
  (9, 10),
  (9, 11),
  (9, 12),
  (10, 12),
  (10, 13),
  (10, 14),
  (10, 15)],
 [(10, 3),
  (11, 3),
  (11, 4),
  (11, 5),
  (11, 6),
  (11, 7),
  (10, 7),
  (9, 7),
  (9, 8),
  (9, 9),
  (9, 10),
  (9, 11),
  (9, 12),
  (9, 13),
  (10, 13),
  (10, 14),
  (10, 15)],
 [(10, 3),
  (11, 3),
  (11, 4),
  (11, 5),
  (11, 6),
  (11, 7),

In [232]:
testpaths.sort(key= lambda path: (path[1][1],path[1][0]))

In [233]:
testpaths

[[(10, 3),
  (11, 3),
  (11, 4),
  (11, 5),
  (10, 5),
  (9, 5),
  (9, 6),
  (9, 7),
  (9, 8),
  (9, 9),
  (9, 10),
  (9, 11),
  (9, 12),
  (10, 12),
  (10, 13),
  (10, 14),
  (10, 15)],
 [(10, 3),
  (11, 3),
  (11, 4),
  (11, 5),
  (10, 5),
  (9, 5),
  (9, 6),
  (9, 7),
  (9, 8),
  (9, 9),
  (9, 10),
  (9, 11),
  (9, 12),
  (9, 13),
  (10, 13),
  (10, 14),
  (10, 15)],
 [(10, 3),
  (11, 3),
  (11, 4),
  (11, 5),
  (10, 5),
  (9, 5),
  (9, 6),
  (9, 7),
  (9, 8),
  (9, 9),
  (9, 10),
  (9, 11),
  (9, 12),
  (9, 13),
  (9, 14),
  (10, 14),
  (10, 15)],
 [(10, 3),
  (11, 3),
  (11, 4),
  (11, 5),
  (11, 6),
  (11, 7),
  (10, 7),
  (9, 7),
  (9, 8),
  (9, 9),
  (9, 10),
  (9, 11),
  (9, 12),
  (10, 12),
  (10, 13),
  (10, 14),
  (10, 15)],
 [(10, 3),
  (11, 3),
  (11, 4),
  (11, 5),
  (11, 6),
  (11, 7),
  (10, 7),
  (9, 7),
  (9, 8),
  (9, 9),
  (9, 10),
  (9, 11),
  (9, 12),
  (9, 13),
  (10, 13),
  (10, 14),
  (10, 15)],
 [(10, 3),
  (11, 3),
  (11, 4),
  (11, 5),
  (11, 6),
  (11, 7),

In [236]:
testpaths[0][1]

(11, 3)

In [604]:
turns

37